In [27]:
# 라이브러리 import
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [28]:
boston = load_boston()
df_x = pd.DataFrame(boston.data, columns=boston.feature_names)
df_y = pd.DataFrame(boston.target, columns=['MEDV'])

In [29]:
df_x

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [30]:
df_y

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


In [31]:
"""
1. 전체 데이터를 학습 데이터와 검증 데이터로 나누어 학습 MSE와 검증 MSE를 계산하세요.
학습 모형은 일반 선형 회귀 모형을 사용하세요.
이 때, 학습 데이터와 검증 데이터의 비율은 0.8 : 0.2 로 나누어 주세요.
"""

'\n1. 전체 데이터를 학습 데이터와 검증 데이터로 나누어 학습 MSE와 검증 MSE를 계산하세요.\n학습 모형은 일반 선형 회귀 모형을 사용하세요.\n이 때, 학습 데이터와 검증 데이터의 비율은 0.8 : 0.2 로 나누어 주세요.\n'

In [32]:
# Numpy array로 변형
X = df_x.to_numpy()
y = df_y.to_numpy().squeeze(axis=1)
print(X.shape)
print(y.shape)

(506, 13)
(506,)


In [33]:
# 독립변수 상수항 추가
X_aug = sm.add_constant(X, prepend=False)
print(X_aug.shape)

# index 생성
idx = np.arange(X.shape[0])

# 학습 데이터/검증 데이터 분리
train_idx, test_idx = train_test_split(idx, train_size = num_train, test_size=0.2)
train_X_aug, test_X_aug = X_aug[train_idx,:], X_aug[test_idx,:]
train_y, test_y = y[train_idx], y[test_idx]

# 선형 회귀 모형 추정
mod = sm.OLS(train_y, train_X_aug).fit()
print(mod.summary())

(506, 14)
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     15.25
Date:                Mon, 20 Jan 2020   Prob (F-statistic):           2.77e-17
Time:                        14:45:19   Log-Likelihood:                -298.48
No. Observations:                 101   AIC:                             625.0
Df Residuals:                      87   BIC:                             661.6
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1440      0.182     -0.79

In [34]:
# 예측값 계산
train_y_pred = mod.predict(train_X_aug)
test_y_pred = mod.predict(test_X_aug)
# MSE 계산
print("Train MSE", mean_squared_error(train_y, train_y_pred))
print("Test MSE", mean_squared_error(test_y, test_y_pred))

Train MSE 21.598135529140894
Test MSE 30.161335820626395


In [35]:
"""
위에서 나눈 학습 데이터와 검증 데이터를 이용하여 Lasso를 이용해 학습 MSE와 검증 MSE를 계산하세요.
이 때, Lasso의 hyper-parameter인 alpha 값으로 아래 주어진 리스트 모두에 대해 MSE를 확인해보고 가장 좋은 결과를 이용하세요.
"""

'\n위에서 나눈 학습 데이터와 검증 데이터를 이용하여 Lasso를 이용해 학습 MSE와 검증 MSE를 계산하세요.\n이 때, Lasso의 hyper-parameter인 alpha 값으로 아래 주어진 리스트 모두에 대해 MSE를 확인해보고 가장 좋은 결과를 이용하세요.\n'

In [36]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold

In [37]:
# 학습 데이터/검증 데이터 분할
cv = KFold(n_splits=5, shuffle=True)
idx = np.arange(X.shape[0])
train_idx, test_idx = train_test_split(idx, test_size=0.2)
X_train, y_train = X[train_idx, :], y[train_idx]
X_test, y_test = X[test_idx, :], y[test_idx]

In [38]:
alpha_list = [1, 0.5, 0.2, 0.1, 0.05, 0.01, 0.005]

for a in alpha_list:
    # Lasso 회귀 모형 선언 및 학습 데이터에 대해 모형 추정
    print("Alpha: ", a)
    mse_train_list = []
    mse_test_list = []
    # 학습 데이터/검증 데이터에 대한 예측값 계산
    for train_idx, test_idx in cv.split(X):
        X_train, y_train = X[train_idx, :], y[train_idx]
        X_test, y_test = X[test_idx, :], y[test_idx]

        # 학습 MSE/검증 MSE 계산
        lasso_reg = Lasso(alpha = a)
        lasso_reg.fit(X_train, y_train)

        y_pred_train = lasso_reg.predict(X_train)
        y_pred_test = lasso_reg.predict(X_test)

        mse_train = mean_squared_error(y_train, y_pred_train)
        mse_test = mean_squared_error(y_test, y_pred_test)

        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)

    print("MSE train (lasso): ", np.mean(mse_train_list))
    print("MSE test (lasso): ", np.mean(mse_test_list))
    print()

Alpha:  1
MSE train (lasso):  26.694196414398714
MSE test (lasso):  28.00640494937695

Alpha:  0.5
MSE train (lasso):  23.942051337866076
MSE test (lasso):  25.97983299075265

Alpha:  0.2
MSE train (lasso):  23.220433822344855
MSE test (lasso):  24.97250017512695

Alpha:  0.1
MSE train (lasso):  22.741301282684457
MSE test (lasso):  25.92171054244203

Alpha:  0.05
MSE train (lasso):  22.51920932765521
MSE test (lasso):  24.850438779991237

Alpha:  0.01
MSE train (lasso):  21.728424745664437
MSE test (lasso):  23.810181612860383

Alpha:  0.005
MSE train (lasso):  21.533376246163247
MSE test (lasso):  25.534246714905404

